# Imports

In [1]:
import random
import secrets
from decimal import *

# Functions

Function to return GCD between 2 numbers (a and b)

In [2]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

Rabin-Miller Primality Test<br>
Checks if a number is prime, n is the number to be tested, s is the number of rounds<br>

In [3]:
def RMPT(n, s):
    q = n - 1
    k = 0
    
    # Set n = 2^k*q, this while loop returns values k and q
    while q % 2 != 1:
        q //= 2
        k += 1
    
    # Number of iteration depends on security parameter S
    for _ in range(s):
        
        # Choose a witness
        a = random.randrange(2, n - 2)      
        
        # Set z = a^q mod n
        z = pow(a, q, n)
        
        # If z = 1 mod n or z = -1 mod n, then n is likely prime.
        if z == 1 or z == n - 1:
            continue #Not a witness, pick another a
        
        # Else, look through i=0 to k-1
        for i in range(1, k):
            # Square z: z = a^2 mod n
            z = pow(a, pow(2, i)*q, n)
            
            # If z = -1 mod n, then n is likely prime
            if z == n - 1:
                break #Not a witness, pick another a
                
        # After executing for loop, if a^((2^K)*Q)modP != -1 for all values of K, then a is a witness, therefore n is definitely not prime
        else:
            return False  
        
    return True #After S iterations still not a composite, chance to be a prime

Function to convert a string to its ascii value

In [4]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3) # Padding
        ascii_message += ascii_char
    return int(ascii_message)

Function to convert an ascii input to its string value

In [5]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

Algorithm to generate decryption exponent from parameters e, p, and q

In [6]:
def RSA_generate_D(e, p, q):
    N = p * q
    phi_N = (p - 1) * (q - 1)
    assert gcd(e, phi_N) == 1, "GCD of e and phi N must be 1"
    d = pow(e, -1, phi_N)
    return d

Algorithm to encrypt a message by raising it to the encryption exponent e mod N

In [7]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

Algorithm to decrypt a ciphertext by raising it to the decryption exponent d mod N

In [8]:
def RSA_decrypt(encrypted_message, d, N):
    ascii_message = pow(encrypted_message, d, N)
    return ascii_message

Continuously generates random bit_length numbers<br>
If number is even, add one to make it odd<br>
Tries to factorize the number with small primes to determine if composite. This helps to reduce expensive computation<br>
Uses Rabin-Miller Primality Test if it is still possible to be a prime number

In [9]:
def Generate_Prime(bit_length, RMPT_rounds):   
    while(True):
        num = secrets.randbits(bit_length)
        
        #If number is even, make it odd
        if num%2 == 0:
            num += 1
        #Try factorize using small primes
        elif num%3 == 0:
            continue
        elif num%5 == 0:
            continue
        elif num%7 == 0:
            continue
        elif num%11 == 0:
            continue
        #Try the expensive Rabin-Miller Primality Test
        elif RMPT(num, RMPT_rounds):
            print(num)
            return num

# Using RSA 1024 for signatures

Generate 2 random 1024 bit primes

In [10]:
p = Generate_Prime(1024, 3)

150869311047398355087557418398554697373072229373354303345728941183834038766042944149622243250847000808600476374709165706532952644404660338905451468500973210545801172533476278790631261397525706216062685292571484237020987735836914236538073344387289066471527748642520157231242910318468580766333660714323946056919


In [11]:
q = Generate_Prime(1024, 3)

101194771083320368488165550016208718687594418031270340345476626329557430561611325765635842490734339164296760233814284675508472942250893590006557890846503899384591065012762549356026216920677493496923308556777691454735677155206636086729357179334617849917882398441845934415929094498411029928863384186609996448831


In [12]:
N = p * q
print(N)

15267185394939733278886954500702577955511119879952307307469514141070620177030056107791608564602042287415821667358078796878009843662556899880054581142129235052210725435288867484005327100190660566134075746952189620710160384902327812223115396593455337766445109497499856662594052960804405056474726630154306534990398583579000704726511042472034988971343046282765202411466118317399543965370355076066400122175032032647739070341988923330695027762189285186225098073997411749486626753021567634905553379638385777674197430613828120190226518630303513063576808525459938703978708610931705376029723250062100813890942352601394997011689


Using 17 as our encryption exponent, generate d

In [13]:
e = 17
d = RSA_generate_D(e, p, q)

In [14]:
print(d)

4490348645570509787907927794324287633973858788221266855138092394432535346185310619938708401353541849239947549222964352022944071665457911729427817982979186780061978069202608083530978558879606048862963454985938123738282466147743474183269234292192746401895620440441134312527662635530707369551390185339501922055925446910844113530275093971653007516259523422164875814051444926437103675306676707691512363656897262551424068745136903808427530051627568016856790798426451364575363092786861413752616441564759581753297481401317336617197015805664694923620405294628832878702734842308040965994868013307418000939925090500135604266453


Lets say Eliezer wants to send a message, but he wants to make sure that others can be sure he was the one that truly sent the message

In [15]:
message = "Hi from Eliezer!"

ascii_message = convert_to_ascii(message)
print(ascii_message)

72105032102114111109032069108105101122101114033


Eliezer can encrypt the message using his private key d

In [16]:
encrypted_message = RSA_encrypt(ascii_message, d, N)

In [17]:
print(encrypted_message)

304820283272003699192890561125403875712724615032284634610698850932889801212774653865312869228197778162092194433606570940808751296040310034890414022525959896545428386626038670647736161798499216719676405231447226778252831907262986370318319019378960303321736531854447919350014101633930451284365677074035073936236988588362562158832174553383989062334819624990435648933736502698308201494759916467283082166381871240630985475513508305405790056820661894547968817618546008777856163788031545738529457589601196051825114280211822452403238784789704711585114476770432765472403216301464462053100702688922775635617249569117645401620


In [18]:
print(convert_to_string(encrypted_message))

İ̴ěĐʻÀͺȱ}Ɠͫˈ˔ɧ Ĝɺɢʺ͒Τ͹̡Ô̆ʍ͡ĸͥäÅ̊¢\ÂƱɞȺά̨˯Ĩ(Ķ"ͺƞȍο΀ȡƬƂɲ&ʞʇˠ¡̞ǳØˏʤƕçƿâ̊ü̿΋ĆϚŲľĿźπįŁˠȓ͖ƿΗŞeɹ΢ǃĜŭʥJ#IΨìϜɌŪȲ̀®ȩſϝ>Ŏ̳ɰϞƳʈΥˠǶʺĴÉǮ˷ΔǓěR¦ŽͧðɶϙǛȁǼıƕ̖8̴ʕ;ȣψ̱ɪȢ̉͘£̔ȡˢȑǉɍəÄ3̹rĘÓ̶ǄƓî̐̕ˀˇɉrǜ̂ư˽ǘƓØĭǐǎ5dʾʰΚ̇ɻɩùȹuʅƑɬ


Now anyone that wants to verify if the message was really from Eliezer can use his public key to decrypt the message

In [19]:
decrypted_message = RSA_decrypt(encrypted_message, e, N)
print(convert_to_string(decrypted_message))

Hi from Eliezer!


# Blinding Attack

An attacker can execute what is known as a Blinding attack which will allow the attacker to obtain a message encrypted with the victim's private key.
This message is something that the victim would never knowingly sign.

In [20]:
bad_message = "Transfer $1000000 to Joshua"
bad_ascii_message = convert_to_ascii(bad_message)
print(bad_ascii_message)

84114097110115102101114032036049048048048048048048032116111032074111115104117097


First, attacker needs to choose a random number, such that (R^e*M) mod N is something the victim will knowingly sign - Random number will be 7 in this case.

In [21]:
random_number = 7
bad_ascii_message = bad_ascii_message * pow(random_number,e)
print(bad_ascii_message)

19567505644295919156614934743979004764790938642435431729180738597609546366459820295770887978079


In [22]:
print(convert_to_string(bad_ascii_message))

ȷǹʄħΗɦΦ˧ϓ˼̖ΪʂƳƯ˙´ˢɕɡȢŮǋ̴ħ̂ͷϒO


If the attacker somehow manages to get the recipient to sign it, the attacker will be able to obtained the bad message signed with the victim's private key.

In [23]:
bad_encrypted_message = RSA_encrypt(bad_ascii_message, d, N)
print(bad_encrypted_message)

14946988574408187690930852874559302028214741448953531142735851916064612268432196292930750229412438333936855176555034128883943195631031528769427521182356214990982741142580573640009484651988371681004142779244934542386575326052351275461588442962518997136744141162613947058736730476821317829140714496140154311063430452448520518224323738355952038246415676355891843055985151858399819814587925697657427148120120655339073413525521516400285804285718908683617100698578884161142003312257975905356554119100219163866434116075026879151460886705555109699406788495618243178949710370037204188756421746585328949090400511663646307775414


When the victim signs the bad message, it becomes (R^e*M)^d. This allows attacker to retrieve the signed message by dividing the signed message by R=7 and then mod N - Divide signed message by R to get M^d

In [24]:
signed_message = (bad_encrypted_message * pow(7, -1, N)) % N
print(signed_message)

8678363537032483932513102339523862270678300155544350437877770619896638971360337802329367989031223599454902882661324359931138960945528889772798752086963417163944988206921025156003637993222907625629481431443071914931008068679904958875847804677555001490868495665016216720931269908462076142652127769514724845147803743312217518914836695110293857880063545029169635755769072401514064530099855846550946787806459536183184374935926898056052983938898109177470342131510159915657411938760382687153316322573625213841289486845215891388877206085209378412876744867428294184412262314690331473835084499538804484394746795638261614115783


Decrypt with public key, and we see that this message was indeed signed by eliezer!

In [25]:
decrypted_message = RSA_decrypt(signed_message, e, N)
print(convert_to_string(decrypted_message))

Transfer $1000000 to Joshua


# Conclusion

To protect against this attack, the signer can pad the message after receiving it but before signing it

Let f be some sort of padding function,

m'= m * r^e mod N to be the tampered message

The signer pads the m', m'' = f(m') and then signs it, (m'')^d mod N

If the attacker tries to unblind, they will not be able to get the signed message

In this example, we will reuse our bad ascii message from the above demonstration

In [26]:
bad_ascii_message

19567505644295919156614934743979004764790938642435431729180738597609546366459820295770887978079

Legitimate receiver and signer decide on a secret fixed padding before hand, assume padding is "iLoveCrypto". HOWEVER, best case would be to use a padding algorithm

In [27]:
padding = convert_to_ascii("iLoveCrypto")

In [28]:
padded_bad_ascii_message = bad_ascii_message + padding

In [29]:
padded_bad_ascii_message

19567505644295919156614934743979004764790938642435431729180738702685657484560887409892000094190

Sign the padded message

In [30]:
bad_encrypted_message = RSA_encrypt(padded_bad_ascii_message, d, N)
print(bad_encrypted_message)

13925348472104159651116936645931814267059966097741163783799078532738478300832283553459610604747961767645610355348128930671769806592239998460829558782851456106445719440910290145984966806001864392743527683356060053153605448881782367890106868468571785946864779301288937751435009429744473706291371201832565642560798375810324857412941264456374606392845128321901604965973325563157689911050484010182478002000587941097534677933721190674632341038807151092153875902004690728580046370077270977772328919178684381716865196315868032361319995375764712624243159997751802498982155104782193926598429242927587962609151836651567190212053


Then, the attacker tries retrieve the signed message by dividing the signed message by 7 and then mod N - Divide signed message by R to get M^d

In [31]:
signed_message = (bad_encrypted_message * pow(7, -1, N)) % N
print(signed_message)

12894467920971832292221672735634957720659366499643242903020949891155939883711794870345379061108310457817816956019788987865974146415003499694443209213356804481071335231050661080858800329565023889059129488302429736672058196199060204143669121633692639254155760969826888723486467747680928426952143478943442616787541613386475483006499496688078507321365765676532516717614845307165058533986037055787782658982249729190890004234809401046872497121393368146182766610284535639430454305026444164614299402481516181441121764197858376187493226932468896848875314660721642288410814022777245829535292213319727433151980514236934596467214


In [32]:
decrypted_message = RSA_decrypt(signed_message, e, N)
print(convert_to_string(decrypted_message))

²ǆȽèɒƦŀȍƉöĘÏ̚ƗɲͿϓŃ΃ĜφĨ˖ʎ¢̗ęʑοɈϕʮ¶ƎȿƵΊǭΰƮȦƈƯȀǮ!ň~ʎΰʕɌɹȺZėɥž͚ɀťκɇīɾΖˇÂ¶ήìʕǄͳǶ(úɣĩIɋ˼ώ̔ǳʫ̺Ĥ˂Ϗ"˘úïǸȩȝ(đŖ˃ʕɜ¼Ƌϕ̦͜ΫÚʿïʼζ§̧̒ʽͣ˒Ɲ˪@Ƴ<ś˶ʑƞ̟àȁTʹʹkʈǶħɥɚʩμġɬʧŪɈûȸæàȫ͇ŨΫƷƎƩͺ˻̡.ƃļĠȢʷȵˮŎʹtƶƂʊŷɢ˨ʂǞ5ɵƬʺȾő̡ͦ˞˪Ã«
